This notebook aims to test different registration methods using the Ground Truth points that were manually detected on EM and LM images.

#### Loading the Ground truth files in XML

In [49]:
import os
import numpy as np
import bigfish.stack as stack
import bigfish.plot as plot
from utils import xml_to_dataframe


Load the locations for fiducial particles in EM and LM. The locations were manually detected using ICY software and were saved in XML format. These point locations are considered to be the ground truth. For further processing we need to load the XML format and convert it into Pandas dataframe.

In [ ]:
#target_xml = 'Ground_truth_fiducials_EM.xml'
target_xml = 'Ground_truth_fiducials_EM_only_clusters.xml'
target_df = xml_to_dataframe(target_xml)
print(target_df)

#source_xml = 'Ground_truth_fiducials_LM.xml'
source_xml = 'Ground_truth_fiducials_LM_only_clusters.xml'
source_df_small = xml_to_dataframe(source_xml)
print(source_df_small)

EM and LM images have most probably different sizes. LM image is usually much smaller. The points that were detected in LM image coordinate system needs to be rescaled to EM image coordinate system, otherwise it might be very hard with the registration.

In [52]:
def find_the_scale(EM_shape, LM_shape):
    scale_x = EM_shape[0]/LM_shape[0]
    scale_y = EM_shape[1]/LM_shape[1]
    return scale_x, scale_y

In [ ]:
# load the EM and LM images

input_path = "E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1"

EM_image_path = os.path.join(input_path, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM.tif")
LM_image_path  = os.path.join(input_path, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_LM.tif")

EMimage = stack.read_image(EM_image_path)
LMimage_small = stack.read_image(LM_image_path)


# Find what is the scaling rate between the 2 images
scale_x, scale_y = find_the_scale(EMimage.shape, LMimage_small.shape)

print("Scale x: ",scale_x)
print("Scale y: ",scale_y)


# Resize the LM point positions
source_df = xml_to_dataframe(source_xml)
source_df['pos_x'] = source_df_small['pos_x']*scale_x
source_df['pos_y'] = source_df_small['pos_y']*scale_y


# Resize the LM image to fit the position of the resized points
LMimage = stack.resize_image(LMimage_small, EMimage.shape, method='bilinear')

#scale_x = 24.873456790123456
#scale_y = 22.966165413533833

Converting the Panda dataframe into numpy array (2D points (X,Y) and 3D points (X,Y,Z)). The 3D points are needed if we plan to convert them to point clouds later

In [54]:
# Converting dataframe into numpy array of coordinate pairs (X,Y) 
target = target_df[['pos_x', 'pos_y']].to_numpy()
source = source_df[['pos_x', 'pos_y']].to_numpy()
source_small = source_df_small[['pos_x', 'pos_y']].to_numpy()

# Adding the Z-dimension to the 2D points
target3D = np.hstack((target, np.zeros((len(target), 1))))
source3D = np.hstack((source, np.zeros((len(source), 1))))
source3D_small = np.hstack((source_small, np.zeros((len(source_small), 1))))

Plotting the point coordinates on top of the image to double check the correct placement

In [ ]:
#plot.plot_detection(LMimage_small[:,:,1], source_small, contrast=True)
plot.plot_detection(LMimage[:,:,1], source, shape="circle", radius = 3*scale_y, color = "red", linewidth = 1, fill=False, contrast=True) 
plot.plot_detection(EMimage, target, radius = 3*scale_y, contrast=False)

Converting points in numpy array (X,Y,Z) into points clouds and saving them as .ply files

In [ ]:
import open3d as o3d

# Convert NumPy array to Open3D PointCloud
target_pcd = o3d.geometry.PointCloud()
target_pcd.points = o3d.utility.Vector3dVector(target3D)
source_pcd = o3d.geometry.PointCloud()
source_pcd.points = o3d.utility.Vector3dVector(source3D)

# Visualize the point cloud
#o3d.visualization.draw_geometries([target_pcd])
#o3d.visualization.draw_geometries([source_pcd])

# Assuming 'point_cloud' is your Open3D PointCloud object
o3d.io.write_point_cloud("target.ply", target_pcd)
o3d.io.write_point_cloud("source.ply", source_pcd)

### Testing different registration methods

Loading source and target point clouds

In [ ]:
import open3d as o3d
target_pcd = o3d.io.read_point_cloud('target.ply')
source_pcd = o3d.io.read_point_cloud('source.ply')

Registration method: Coherent point drift

In [4]:
from probreg import cpd

# Perform CPD registration
tf_param, _, _ = cpd.registration_cpd(source_pcd, target_pcd)

# Apply the transformation to the source point cloud
transformed_source_cpd = tf_param.transform(source_pcd.points)

# Visualize the results
source_pcd.paint_uniform_color([1, 0, 0])  # Red for source
target_pcd.paint_uniform_color([0, 1, 0])  # Green for target
transformed_source_cpd_pcd = o3d.geometry.PointCloud()
transformed_source_cpd_pcd.points = o3d.utility.Vector3dVector(transformed_source_cpd)
transformed_source_cpd_pcd.paint_uniform_color([0, 0, 1])  # Blue for result

o3d.visualization.draw_geometries([source_pcd, target_pcd, transformed_source_cpd_pcd])

Testing different registration methods provided by Probreg package

In [47]:
# Visualize results
def visualize_result_nparray(source, target, result, title):
    source_cloud = o3d.geometry.PointCloud()
    source_cloud.points = o3d.utility.Vector3dVector(source)
    source_cloud.paint_uniform_color([1, 0, 0])  # Red

    target_cloud = o3d.geometry.PointCloud()
    target_cloud.points = o3d.utility.Vector3dVector(target)
    target_cloud.paint_uniform_color([0, 1, 0])  # Green

    result_cloud = o3d.geometry.PointCloud()
    result_cloud.points = o3d.utility.Vector3dVector(result)
    result_cloud.paint_uniform_color([0, 0, 1])  # Blue

    o3d.visualization.draw_geometries([source_cloud, target_cloud, result_cloud], window_name=title)

def visualize_result_pcd(source, target, result, title):
    source.paint_uniform_color([1, 0, 0])  # Red
    target.paint_uniform_color([0, 1, 0])  # Green
    result.paint_uniform_color([0, 0, 1])  # Blue
    o3d.visualization.draw_geometries([source, target, result], window_name=title)

# Print transformations
def print_transformations(transformation_paramters, title):
    print(title)
    print(transformation_paramters.rot)  # Rotation matrix
    print(transformation_paramters.t)    # Translation vector
    print(transformation_paramters.scale)  # Scale factor

    # Evaluate alignment using chamfer distance
def chamfer_distance(A, B, title):
    distances = np.min(np.sum((A[:, np.newaxis, :] - B[np.newaxis, :, :]) ** 2, axis=2), axis=1)
    print(f"Chamfer distance = {np.mean(distances)} ({title}) ")

    return np.mean(distances)

In [ ]:
import numpy as np
import open3d as o3d
from probreg import cpd, gmmtree, filterreg, bcpd, l2dist_regs

# Load or create your point cloud data
source = o3d.io.read_point_cloud("source.ply")
target = o3d.io.read_point_cloud("target.ply")

# Convert to numpy arrays
source_points = np.asarray(source.points)
target_points = np.asarray(target.points)

# 1. Coherent Point Drift (CPD)
# tf_param_cpd, _, _ = cpd.registration_cpd(source_points, target_points)
# result_cpd = tf_param_cpd.transform(source_points)

# 1. Coherent Point Drift (CPD)
## Rigid CPD - WORKS
tf_param_rigid, result_rigid2, result_rigid3 = cpd.registration_cpd(source_points, target_points, tf_type_name='rigid', maxiter=500, tol=1e-5)
result_rigid = tf_param_rigid.transform(source_points)

'''
## Affine CPD - Error - LinAlgError: Singular matrix
#tf_param_affine, _, _ = cpd.registration_cpd(source_points, target_points, tf_type_name='affine')
#result_affine = tf_param_affine.transform(source_points)
#print("Affine CPD Transformation:")
#print(tf_param_affine.b)  # Affine matrix
#print(tf_param_affine.t)  # Translation vector

acpd = cpd.AffineCPD(source, use_cuda=False)
tf_param_affine, result_nonrigid2, _ = acpd.registration(target)
result_nonrigid = tf_param_affine.transform(source_points)
print("Affine CPD Transformation:")
print(tf_param_affine.b)  # Affine matrix
print(tf_param_affine.t)  # Translation vector

## Non-rigid CPD - WORKS, but NO TRANSFORMATION
tf_param_nonrigid, _, _ = cpd.registration_cpd(source_points, target_points, tf_type_name='nonrigid', maxiter=200, tol=1e-5)
result_nonrigid = tf_param_nonrigid.transform(source_points)
print("Non-rigid CPD Transformation:")
print(tf_param_nonrigid.g)  # Displacement field
print(tf_param_nonrigid.w)  # Weight matrix
'''

# 2. FilterReg - WORKS
tf_param_filterreg, _, _ = filterreg.registration_filterreg(source_points, target_points)
result_filterreg = tf_param_filterreg.transform(source_points)
#print('RESULTS:', result_filterreg)
#print('RESULTS2:',result_filterreg2)

'''
# 3. GMMReg (Gaussian Mixture Model Registration) Error - TypeError: cannot unpack non-iterable RigidTransformation object
#tf_param_gmmreg, _ = l2dist_regs.registration_gmmreg(source_points, target_points)
#result_gmmreg = tf_param_gmmreg.transform(source_points)

# 4. SVR (Support Vector Registration) Error - InvalidParameterError: The 'gamma' parameter of OneClassSVM must be a str among {'scale', 'auto'} or a float in the range [0.0, inf). Got np.float64(inf) instead.
#tf_param_svr, _ = l2dist_regs.registration_svr(source_points, target_points)
#result_svr = tf_param_svr.transform(source_points)

# 5. GMMTree (Hierarchical Stochastic Model) - WORKS, but NO TRANSFORMATION
tf_param_gmmtree, _ = gmmtree.registration_gmmtree(source_points, target_points)
result_gmmtree = tf_param_gmmtree.transform(source_points)

# 6. Bayesian Coherent Point Drift (BCPD) (experimental) - Error: cannot unpack non-iterable CombinedTransformation object
#tf_param_bcpd, _ = bcpd.registration_bcpd(source_points, target_points)
#result_bcpd = tf_param_bcpd.transform(source_points)
'''

# Visualize results for each method
#visualize_result(source_points, target_points, result_cpd, "CPD Registration")
visualize_result_nparray(source_points, target_points, result_rigid, "Rigid CPD")
#visualize_result(source_points, target_points, result_affine, "Affine CPD")
#visualize_result(source_points, target_points, result_nonrigid, "Non-rigid CPD")
#visualize_result(source_points, target_points, result_gmmreg, "GMMReg")
#visualize_result(source_points, target_points, result_svr, "SVR")
#visualize_result(source_points, target_points, result_gmmtree, "GMMTree")
visualize_result_nparray(source_points, target_points, result_filterreg, "FilterReg")
#visualize_result(source_points, target_points, result_bcpd, "BCPD")

# Print transformations
print_transformations(tf_param_rigid, "Rigid CPD Transformation:")
print_transformations(tf_param_filterreg, "FilterReg Transformation:")
#print_transformations(tf_param_gmmreg, "GMMReg Transformation:")
#print_transformations(tf_param_svr, ""SVR Transformation:"")
#print_transformations(tf_param_gmmtree, "GMMTree Transformation:")
#print_transformations(result_bcpd, "BCPD Transformation:")

Open3D library that provides ICP (Iterative Closest Point) registration algorithms - point-to-point:

In [ ]:
import open3d as o3d
import numpy as np
import copy

# Load or create your point cloud data and initial alignment
source = o3d.io.read_point_cloud("source.ply")
target = o3d.io.read_point_cloud("target.ply")
threshold = 10000
trans_init = np.identity(4)

# Point-to-point ICP
print("Applying point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp( source, target, threshold, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint())

print(reg_p2p)
print("Transformation is: \n", reg_p2p.transformation)

result_p2p = copy.deepcopy(source)
result_p2p.transform(reg_p2p.transformation)

# Visualize the result
visualize_result_pcd(source, target, result_p2p , "point-to-point ICP")



In [ ]:
chamfer_distance(target_points, source_points, "No transform")
chamfer_distance(target_points, result_rigid,"Rigid CPD Transformation")
chamfer_distance(target_points, result_filterreg, "FilterReg Transformation")
chamfer_distance(target_points, np.asarray(result_p2p.points), "point-to-point ICP")
